## 1. Load the data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import plotly.express as px

In [3]:
# Load movements
movements2023 = pd.read_csv('../data/2023_verkehrszaehlungen_werte_fussgaenger_velo.csv')

# Load metadata
meta = pd.read_csv('../data/GeoData_StadtZurich/data/taz.view_eco_standorte.csv')

# Load weather data
weather = pd.read_csv('../data/ugz_ogd_meteo_h1_2023.csv')

In [4]:
weather.head()

,Datum,Standort,Parameter,Intervall,Einheit,Wert,Status
0,2023-01-01T00:00+0100,Zch_Stampfenbachstrasse,T,h1,°C,11.57,provisorisch
1,2023-01-01T00:00+0100,Zch_Stampfenbachstrasse,Hr,h1,%Hr,72.29,provisorisch
2,2023-01-01T00:00+0100,Zch_Stampfenbachstrasse,p,h1,hPa,971.62,provisorisch
3,2023-01-01T00:00+0100,Zch_Stampfenbachstrasse,RainDur,h1,min,0.00,provisorisch
4,2023-01-01T00:00+0100,Zch_Stampfenbachstrasse,StrGlo,h1,W/m2,0.01,provisorisch


## Prepare Geograhical Data

In [15]:
def extract_coordinates(point_str):
    # Use regular expression to find numbers within parentheses
    x, y = map(float, re.findall(r"POINT \(([\d\.]+) ([\d\.]+)\)", point_str)[0])
    return x, y

# Extracting the coordinates for each point
meta['coordinates'] = [extract_coordinates(point) for point in meta['geometry']]

# Split coordinates into latitude and longitude
meta['lat'], meta['lon'] = [lon for lon, lat in meta['coordinates']], [lat for lon, lat in meta['coordinates']]

In [20]:
px.set_mapbox_access_token(open(".mapbox_token").read())
fig = px.scatter_mapbox(meta, lat="lat", lon="lon", #color="peak_hour", size="car_hours",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10)
fig.show()


FileNotFoundError: [Errno 2] No such file or directory: '.mapbox_token'

In [31]:
# Extract the x and y coordinates
def extract_coordinates(point_str):
    # Cut of 'POINT '
    extract = point_str[6:]
    return extract

meta['coordinates'] = meta['geometry'].apply(extract_coordinates)
meta

,abkuerzung,bezeichnung,bis,fk_zaehler,id1,richtung_in,richtung_out,von,objectid,korrekturfaktor,geometry,coordinates
0,VZS_BINZ,Binzmühlestrasse,2.018040e+13,ECO09113500,7,Oerlikon,Glaubtenstrasse,20101130000000,1,1.22,POINT (2681857 1251990.9),(2681857 1251990.9)
1,VZS_BUCH,Bucheggplatz,2.019031e+13,ECO09113505,16,Hofwiesenstrasse,Höngg,20140626000000,2,1.27,POINT (2682647 1250363.9),(2682647 1250363.9)
2,VZS_KLOW,Kloster-Fahr-Weg,2.020020e+13,Y0412032046,53,Stadtgrenze,Innenstadt,20100625000000,3,0.94,POINT (2679028 1250673.9),(2679028 1250673.9)
3,VZS_HOFW,Hofwiesenstrasse,2.019031e+13,ECO09113507,13,Bucheggplatz,---,20100923000000,4,1.28,POINT (2682683.2 1250569.6),(2682683.2 1250569.6)
4,VZS_LIMB,Limmatquai --> Bellevue,2.019060e+13,Y2G14045587,61,Bellevue,---,20140625000000,5,1.47,POINT (2683447 1247062.9),(2683447 1247062.9)
...,...,...,...,...,...,...,...,...,...,...,...,...
168,FZS_LETD,Letten / Dynamo,NaN,U15G3063869,5003,Limmatplatz,Hauptbahnhof,20221012000000,169,0.89,POINT (2682978 1248743.9),(2682978 1248743.9)
169,FZS_HARD,Hardeggsteg,NaN,U15G3063873,5004,Kloster-Fahr-Weg,Hardturmstrasse,20221012000000,170,0.86,POINT (2680439 1249929.9),(2680439 1249929.9)
170,VZS_TOED,Tödistrasse,NaN,Y2H22063445,4255,Seeufer,Hauptbahnhof,20221203000000,171,1.41,POINT (2682881 1246548.9),(2682881 1246548.9)
171,VZS_LIMC,Limmatquai --> Central,NaN,Y2H20083483,4267,Central,NaN,20230727000000,172,1.00,POINT (2683457 1247064.9),(2683457 1247064.9)


In [34]:
# Separate the coordinates into two lists
x_coords, y_coords = zip(meta['coordinates'])

# Step 3: Plot these points on a map (using a simple scatter plot here)
plt.scatter(x_coords, y_coords, c='blue', marker='o')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.title('Plot of Points')
plt.grid(True)
plt.show()

ValueError: too many values to unpack (expected 2)